<a href="https://colab.research.google.com/github/Renatojcalpalhao/Csharp-COMPLETO-2020-Programacao-Orientada-a-Objetos_Projetos/blob/master/Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests, six
import lxml.html as lh
from itertools import cycle, islice
from matplotlib import colors
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

url='http://pokemondb.net/pokedex/all'

#pegou as informações da pagi
page = requests.get(url)

#guardando no doc
doc = lh.fromstring(page.content)

#que estão entre <tr>..</tr> of the site's HTML code
tr_elements = doc.xpath('//tr')

#12 colunas
[len(T) for T in tr_elements[:12]]

tr_elements = doc.xpath('//tr')

#Criando lista vazia
col=[]
i=0

#cabeçalhos
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print '%d:"%s"'%(i,name)
    col.append((name,[]))

    #Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]

    #If row is not of size 10, the //tr data is not from our table
    if len(T)!=10:
        break

    #i é o index da coluna
    i=0

    #
    for t in T.iterchildren():
        data=t.text_content()
        #ver se coluna ta vazia
        if i>0:
        #Convertendo valores para numericos
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

        [len(C) for (title,C) in col]

        Dict={title:column for (title,column) in col}
import pandas as pd
df=pd.DataFrame(Dict)

#organizando o data frame
def str_bracket(word):
    '''Add brackets around second term'''
    list = [x for x in word]
    for char_ind in range(1, len(list)):
        if list[char_ind].isupper():
            list[char_ind] = ' ' + list[char_ind]
    fin_list = ''.join(list).split(' ')
    length = len(fin_list)
    if length>1:
        fin_list.insert(1,'(')
        fin_list.append(')')
    return ' '.join(fin_list)


def str_break(word):
    '''Break strings at upper case'''
    list = [x for x in word]
    for char_ind in range(1, len(list)):
        if list[char_ind].isupper():
            list[char_ind] = ' ' + list[char_ind]
    fin_list = ''.join(list).split(' ')
    return fin_list
    #testando

    word = 'ILovePokemon'
print str_bracket(word)
print str_break(word)

#aplicando no data frame
df['Name']=df['Name'].apply(str_bracket)
df['Type']=df['Type'].apply(str_break)
df.head()

#guardando o data frame usando json

df.to_json('PokemonData.json')

df = pd.read_json('PokemonData.json')
df=df.set_index(['#'])
df.head()

In [ ]:
#analisando o data frame
def max_stats(df, col_list):
    '''Get Pokemon highest value of the column in the Data Frame'''
    message = ''
    for col in col_list:
        stat = df[col].max()
        name = df[df[col]==df[col].max()]['Name'].values[0]
        message += name+' has the greatest '+col+' of '+str(stat)+'.\n'
    return message

def min_stats(df, col_list):
    '''Get Pokemon lowest value of the column in the Data Frame'''
    message = ''
    for col in col_list:
        stat = df[col].min()
        name = df[df[col]==df[col].min()]['Name'].values[0]
        message += name+' has the worst '+col+' of '+str(stat)+'.\n'
    return message

    stats=['Attack', 'Defense','HP', 'Sp. Atk','Sp. Def','Speed','Total']

print max_stats(df, stats)

print min_stats(df, stats)

In [ ]:
#analisando habilidades


#criando novo data frame com outros valores
newDict={}
stats_col=["#","Name","Total","HP","Attack","Defense","Sp. Atk","Sp. Def","Speed"]

#pegando
Dict['Type']=df['Type'].values

#Criando lista vazia
for col in stats_col:
    newDict[col]=[]
    newDict['Type']=[]


#Preenchendo cada valor do dicionário (lista vazia) com dados
for row in range(len(Dict['#'])):
    for t in Dict['Type'][row]:
        for col in stats_col:
            newDict[col].append(Dict[col][row])
        #Anexando o Type separadamente para cada Pokémon no novo dicionário
        newDict['Type'].append(t)

#Converter dicionário em dados
new_df = pd.DataFrame(newDict)

new_df.head()

#tipos exclusivos de Pokémon.
types=new_df['Type'].unique()
print types

In [ ]:
#graficos

my_colors = list(six.iteritems(colors.cnames))
my_colors = list(islice(cycle(my_colors), None, len(new_df)))

def barh_stats():
    '''Plot hbar of mean and std. of each attribute of Pokemon Type'''
    i=0
    plt.figure(figsize=(15,5))
    plt.suptitle('Statistics', fontsize=15)



    #Percorrer cada tipo de Pokémon
    for t in types:

        #Iterar o valor i para mudar a cor para my_colors[i]
        i+=1


        plt.subplot(121)
        plt.title('Mean')
        new_df[new_df['Type']==t].mean().plot(kind='barh', color=my_colors[i])


        plt.subplot(122)
        plt.title('Standard Deviation')
        new_df[new_df['Type']==t].std().plot(kind='barh', color=my_colors[i])

    #Adicionar lista de tipos de Pokémon à legenda
    plt.legend(types,bbox_to_anchor=(1.3, 1.1))

In [ ]:
# usaremos o arquivo PokémonData.json que criamos antes e examinaremos os dados usando uma GUI Python.

In [ ]:
import pandas as pd
import sys, urllib2
import requests
from PyQt4 import QtGui, QtCore

class PokeDex(QtGui.QWidget):

    def __init__(self):
        super(PokeDex, self).__init__()

        self.initUI()

    def initUI(self):
        '''Initial UI'''

        #Grid Layout
        self.grid = QtGui.QGridLayout()
        self.setLayout(self.grid)

        #Parse JSON for DataFrame
        self.df = pd.read_json('PokemonData.json')
        self.df = self.df.set_index(['#'])

        #Drop Down
        self.dropdown = QtGui.QComboBox(self)
        self.names = self.df['Name'].values
        self.dropdown.addItems(self.names)
        self.grid.addWidget(self.dropdown, 0,0,1,1)

        #Search Button
        self.btn = QtGui.QPushButton('Search', self)
        self.btn.clicked.connect(self.runSearch)
        self.grid.addWidget(self.btn, 0,1,1,1)

        #Image
        self.img = QtGui.QLabel(self)
        self.grid.addWidget(self.img, 1,1,1,1)

        #Data
        self.label = QtGui.QLabel(self)
        self.label.setAlignment(QtCore.Qt.AlignCenter)
        self.label.setText('\nName:\n\nType:\n\nHP:\n\nAttack\n\nSp. Attack\n\n Defense:\n\nSp. Defense:\n\nSpeed:\n\nTotal:')
        self.label.setAlignment(QtCore.Qt.AlignLeft)
        self.grid.addWidget(self.label, 1,0,1,1)

        #Customize Widgets
        self.resize(500, 250)
        self.center()
        self.setWindowTitle('PokeDex')
        self.show()

    def runSearch(self):
        '''Event for run button'''

        #Parse value
        index = self.dropdown.currentIndex()
        val= self.names[index]
        cond = self.df['Name']== val

        #Image
        base = 'http://img.pokemondb.net/artwork/'
        img_url = base + val.lower() + '.jpg'
        data = urllib2.urlopen(img_url).read()
        image = QtGui.QImage()
        image.loadFromData(data)
        self.img.setPixmap(QtGui.QPixmap(image))

        #Set values
        name = 'Name:\t\t\t'+val+'\n\n'
        ty = 'Type:\t\t\t'+ ', '.join(self.df[cond]['Type'].values[0])+'\n\n'
        hp = 'HP:\t\t\t'+ str(self.df[cond]['HP'].values[0])+'\n\n'
        atk = 'Attack:\t\t\t'+str(self.df[cond]['Attack'].values[0])+'\n\n'
        satk = 'Sp. Attack:\t\t'+str(self.df[cond]['Sp. Atk'].values[0])+'\n\n'
        deff = 'Defense:\t\t\t'+str(self.df[cond]['Defense'].values[0])+'\n\n'
        sdef = 'Sp. Defense:\t\t'+str(self.df[cond]['Sp. Def'].values[0])+'\n\n'
        speed = 'Speed:\t\t\t'+str(self.df[cond]['Speed'].values[0])+'\n\n'
        total = 'Total:\t\t\t'+str(self.df[cond]['Total'].values[0])+'\n\n'

        #Add text
        final = name+ty+hp+atk+satk+deff+sdef+speed+total
        self.label.setText(final)

    def center(self):
        '''Center Widget on screen'''
        qr = self.frameGeometry()
        cp = QtGui.QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())


def main():
    '''Codes for running GUI'''

    #Create Application object to run GUI
    app = QtGui.QApplication(sys.argv)
    app.aboutToQuit.connect(app.deleteLater)

    #Run GUI
    gui = PokeDex()
    #Exit cleanly when closing GUI
    sys.exit(app.exec_())


if __name__ == '__main__':
    main()